[toc]

# Tensorflow save and restore

## 保存

### 简单的保存

In [1]:
import tensorflow as tf
 
def build_model():
    x = tf.placeholder(tf.float32, [None, 10])
    y = tf.layers.dense(x, 1, activation='sigmoid')
    return y

model = build_model()

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, "mymodel")

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/frame

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/te

```
saver.save(sess, "mymodel")
```

会在当前目录下创建四个文件

```
checkpoint
mymodel.data-00000-of-00001
mymodel.index
mymodel.meta
```

其中，

`mymodel.meta` 保存的是图的结构

### 文件名说明 `.ckpt`

许多博客中都会出现 `model.ckpt` 这样的字样，实际上，`.ckpt` 不是 tensorflow 生成文件中的后缀名，而是用户调用  `saver.save(sess, 'model.ckpt')` 函数时传入的。

在需要使用到 checkpoint 的时候，我们需要传入 checkpoint 的路径，由于 tensorflow 在保存 checkpoint 的时候保存了好几个文件，有可能会让初学者搞混 checkpoint 到底指的是那个文件。实际上，checkpoint 就是我们在 `saver.save` 时传入的参数。

如果我们用

```
saver.save(sess, "model.ckpt")
```

来保存，那么我们的 checkpoint 的路径为 `model.ckpt`，不是 `model.ckpt.data`，更不是 `model.ckpt.meta`。


一般来说，我们不会直接将这四个文件保存在当前目录，而是新建一个目录保存，此时我们可以这样调用

```
saver.save(sess, "saved_model/mymodel")
```

此时会创建 `saved_model`，并在这个目录下生成上述四个文件

### global_step

```
saver.save(sess, "saved_model/mymodel", global_step=100)
```

保存的文件中会添加 global_step，如

```
checkpoint
mymodel-100.data-00000-of-00001
mymodel-100.index
mymodel-100.meta
```

### max_to_keep

参数定义 `saver()` 将自动保存的最近n个ckpt文件，默认n=5，即保存最近的5个检查点ckpt文件。若n=0或者None，则保存所有的ckpt文件。

```
saver = tf.train.Saver(max_to_keep=2)
```

### keep_checkpoint_every_n_hours

与max_to_keep类似，定义每n小时保存一个ckpt文件。

## 载入

载入时有两种方式载入，一种是重新定义再载入的，另一种是不需要重新定义网络结构就可以直接载入的

假设保存时使用的是下列代码

In [2]:
import tensorflow as tf

x = tf.Variable(tf.random_normal(shape=[2,3]), name="x0")
y = tf.Variable(tf.random_normal(shape=[3,2]), name="y0")

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, '/tmp/checkpoint/my_model.ckpt')

In [3]:
!ls /tmp/checkpoint

checkpoint                        my_model.ckpt.index
my_model.ckpt.data-00000-of-00001 my_model.ckpt.meta


### 重新定义结构

先说明没有重新定义网络结构恢复会报错，正确的：

In [4]:
import tensorflow as tf
tf.reset_default_graph()

x = tf.Variable(tf.random_normal(shape=[2,3]), name="x0")
y = tf.Variable(tf.random_normal(shape=[3,2]), name="y0")

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "/tmp/checkpoint/my_model.ckpt")
    print(sess.run(tf.global_variables()))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/checkpoint/my_model.ckpt
[array([[ 1.9626065 , -0.9025703 , -0.3953171 ],
       [-0.24025328, -1.6794065 ,  1.182292  ]], dtype=float32), array([[ 1.7427737 , -0.24296844],
       [ 0.7625185 , -1.6639953 ],
       [ 0.86288357,  0.9713809 ]], dtype=float32)]


如果小小修改一下上面的代码，就报错了，错误的：

```
import tensorflow as tf

x = tf.Variable(tf.random_normal(shape=[2,3])) # 注意这里的x没有命名
y = tf.Variable(tf.random_normal(shape=[3,2]))

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "/tmp/checkpoint/my_model.ckpt")
    print(sess.run(tf.global_variables()))
```

报错信息显示没有找到x变量

```
NotFoundError (see above for traceback): Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:
```

#### tf.train.latest_checkpoint

可以使用 `tf.train.latest_checkpoint()` 来自动获取最后一次保存的模型。

```
model_file = tf.train.latest_checkpoint('tmp/')  # /User/ed/tmp/my_model.ckpt
saver.restore(sess,model_file)
```

### 直接恢复，不重新定义网络结构


利用 `tf.train.import_meta_graph` 来创建saver，而不是 `tf.train.Saver`

假设我们保存时使用的是

```
saver.save(sess, "/tmp/checkpoint/my_model.ckpt")
```

那么我们读入时使用的是

```
import tensorflow as tf

with tf.Session() as sess:
    saver = tf.train.import_meta_graph('/tmp/checkpoint/my_model.meta')
    saver.restore(sess, '/tmp/checkpoint/my_model')
    print(sess.run(tf.global_variables()))
```

## 断点续训

- 只需在初始化后添加一个检查并读取checkpoint的操作即可

```
# ... codes before here

saver = tf.train.Saver()

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    ckpt = tf.train.get_checkpoint_state(SAVE_PATH)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    # ... codes after here
```

# References

- [TensorFlow学习笔记：Saver与Restore - 简书](https://www.jianshu.com/p/b0c789757df6)
- [tensorflow的三种保存格式总结-1(.ckpt) - 知乎](https://zhuanlan.zhihu.com/p/60064947)